# 文章目录
[collections](#1)

[base64](#2)

[struct](#3)

[itertools](#4)

[xml](#5)

[HTMLParser](#6)

[图形编程](#7)

# collections
<a id='1'></a>

collections是Python内建的一个集合模块，提供了许多有用的集合类。

### namedtuple 

namedtuple是一个函数，它用来创建一个自定义的tuple对象，并且规定了tuple元素的个数，并可以用属性而不是索引来引用tuple的某个元素。

这样一来，我们用namedtuple可以很方便地定义一种数据类型，它具备tuple的不变性，又可以根据属性来引用，使用十分方便

In [2]:
#用namedtuple定义一个坐标点
from collections import namedtuple
Point = namedtuple('Point',['x','y'])
p = Point(1,2)

In [3]:
p.x

1

In [4]:
p.y

2

In [5]:
#类似的，要用坐标和半径表示一个园
Circle = namedtuple('Circle',['x','y','r'])

### deque

使用list存储数据时，按索引访问元素很快，但是插入和删除元素就很慢，因为list是线性存储，数据量大的时候，插入和删除效率很低。

deque是为了高效实现插入和删除操作的双向列表，适合用于队列和栈

In [6]:
from collections import deque
q = deque(['a','b','c'])
q.append('x')
q.appendleft('y')
q

deque(['y', 'a', 'b', 'c', 'x'])

###  defaultdict

使用dict的时候，如果引用的Key不村子啊，就会抛出keyError。如果希望key不存在时，返回一个默认值，就可以用defaultdict

In [7]:
from collections import defaultdict
dd = defaultdict(lambda: 'N/A')
dd['key1'] = 'abc'
dd['key1'] #key1存在

'abc'

In [8]:
dd['key2'] #key2不存在

'N/A'

### OrderedDict

使用dict时，key是无序的，在对dict进行迭代时，我们无法确定key的书讯，如果需要保持key的顺序，可以使用OrdereDict

In [12]:
from collections import OrderedDict
d = dict([('f',1), ('c',3), ('b',2)])
d #dict的key是无序的

{'b': 2, 'c': 3, 'f': 1}

In [14]:
od = OrderedDict([('a', 1), ('f', 2), ('c', 3)])
od

OrderedDict([('a', 1), ('f', 2), ('c', 3)])

OrderedDict 的key是按照插入的顺序排列，不是Key本身排序

OrderedDict可以实现一个FIFO的dict，当容量超出限制时，先删除最早添加的Key

In [15]:
from collections import OrderedDict
class LastUpdatedOrderedDict(OrderedDict):
    def __init__(self,capacity):
        super(LastUpdatedOrderedDict,self).__init__()
        self._capacity = capacity
        
    def __setitem__(self,key,value):
        containsKey = 1 if key in self else 0
        if len(self) - containsKey >= self._capacity:
            last = self.popitem(last=False)
            print 'remove:',last
        if containsKey:
            del self[key]
            print 'set:',(key,value)
        else:
            print 'add:',(key,value)
            
        OrderedDict.__setitem__(self,key,value)

### Counter 

是一个简单的计数器，例如，统计字符出现的个数：

In [1]:
from collections import Counter
c = Counter()
for ch in 'programming':
    c[ch] = c[ch]+1
c

Counter({'a': 1, 'g': 2, 'i': 1, 'm': 2, 'n': 1, 'o': 1, 'p': 1, 'r': 2})

# base64
<a id='2'></a>
Base64是一种用64个字符来表示任意二进制数据的方法。

用记事本打开exe、jpg、pdf这些文件时，我们都会看到一大堆乱码，因为二进制文件包含很多无法显示和打印的字符，所以，如果要让记事本这样的文本处理软件能处理二进制数据，就需要一个二进制到字符串的转换方法。Base64是一种最常见的二进制编码方法


In [2]:
#python内置的base64可以直接进行base63的编解码
import base64
base64.b64encode('binary\x00string')

'YmluYXJ5AHN0cmluZw=='

In [3]:
base64.b64decode('YmluYXJ5AHN0cmluZw==')

'binary\x00string'

由于标准的Base64编码后可能出现字符+和/，在URL中就不能直接作为参数，所以又有一种"url safe"的base64编码，其实就是把字符+和/分别变成-和_：

In [4]:
base64.b64encode('i\xb7\x1d\xfb\xef\xff')

'abcd++//'

In [5]:
base64.urlsafe_b64encode('i\xb7\x1d\xfb\xef\xff')

'abcd--__'

In [6]:
base64.urlsafe_b64decode(
'abcd--__')

'i\xb7\x1d\xfb\xef\xff'

由于=字符也可能出现在Base64编码中，但=用在URL、Cookie里面会造成歧义，所以，很多Base64编码后会把=去掉

# struct 
<a id='3'></a>

Python没有专门处理字节的数据类型,但由于str既是字符串，又可以表示字节，所以，字节数组＝str。而在C语言中，我们可以很方便地用struct、union来处理字节，以及字节和int，float的转换。

在Python中，比方说要把一个32位无符号整数变成字节，也就是4个长度的str,你得配合运算符这么写：

In [1]:
n = 10240099
b1 = chr((n & 0xff000000) >> 24)
b2 = chr((n & 0xff0000) >> 16)
b3 = chr((n & 0xff00) >> 8)
b4 = chr(n & 0xff)
s = b1 + b2 + b3 + b4
s

'\x00\x9c@c'

Python提供了一个struct模块来解决str和其他二进制数据类型的转换

In [2]:
import struct
struct.pack('>I',10240099)

'\x00\x9c@c'

'>I'的意思是：>表示字节顺序是big-endian，也就是网络序，I表示4字节无符号整数

In [3]:
#unpack把str变成相应的数据类型
struct.unpack('>IH','\xf0\xf0\xf0\xf0\x80\x80')

(4042322160L, 32896)

根据>IH的说明，后面的str依次变为I：4字节无符号整数和H：2字节无符号整数。
所以，尽管Python不适合编写底层操作字节流的代码，但在对性能要求不高的地方，利用struct就方便多了。

struct模块定义的数据类型可以参考Python官方文档：

https://docs.python.org/2/library/struct.html#format-characters

# hashlib 
Python的hashlib提供了常见的摘要算法，如MD5，SHA1等等。

什么是摘要算法呢？摘要算法又称哈希算法、散列算法。它通过一个函数，把任意长度的数据转换为一个长度固定的数据串（通常用16进制的字符串表示）。

举个例子，你写了一篇文章，内容是一个字符串'how to use python hashlib - by
Michael'，并附上这篇文章的摘要是'2d73d4f15c0db7f5ecb321b6a65e5d6d'。如果有人篡改了你的文章，并发表为'how to use python hashlib - by Bob'，你可以一下子指出Bob篡改了你的文章，因为根据'how to use python hashlib - by Bob'计算出的摘要不同于原始文章的摘要。

可见，摘要算法就是通过摘要函数f()对任意长度的数据data计算出固定长度的摘要digest，目的是为了发现原始数据是否被人篡改过。

摘要算法之所以能指出数据是否被篡改过，就是因为摘要函数是一个单向函数，计算f(data)很容易，但通过digest反推data却非常困难。而且，对原始数据做一个bit的修改，都会导致计算出的摘要完全不同。

In [4]:
#以常见的摘要算法MD5为例，计算出一个字符串里的MD5值
import hashlib

md5 = hashlib.md5()
md5.update('how to use md5 in python hashlib?')
print md5.hexdigest()

d26a53750bc40b38b65a520292f69306


In [5]:
#如果数据量很大，可以分块多次调用update
md5 = hashlib.md5()
md5.update('how to use md5 in ')
md5.update('python hashlib?')
print md5.hexdigest()

d26a53750bc40b38b65a520292f69306


另一个常见的摘要算法是SHA1，SHA1的结果是160bit字节，通常用一个40位的16禁止字符串表示。

比SHA1更安全的是SHA256和SHA512，不过越安全的算法越慢，而且摘要长度更长。

In [6]:
#SHA1
import hashlib

sha1 = hashlib.sha1()
sha1.update('how to use sha1 in python hashlib?')
print sha1.hexdigest()

2c76b57293ce30acef38d98f6046927161b46a44


### 摘要算法应用

举个例子：任何允许用户登录的网站都回存储用户登录的用户名和口令。用户名和口令是存在数据库表中。

如果以明文的形式保存用户口令，如果数据库泄露，所有用户的口令就落入黑客的手里，此外，网站运维人员是可以访问数据库的，也就是能获取到所有用户的口令。

正确的姿势是不存储用户的明文口令，而是存储用户口令的摘要

当用户登录时，首先计算用户输入的明文口令的MD5，然后和数据库存储的MD5对比，如果一致，说明输入口令正确，如果不一致，口令肯定错误

In [7]:
#练习：根据用户输入的口令，计算出存储在数据库中的MD5口令
def calc_md5(password):
    md5 = hashlib.md5()
    md5.update(password)
    
    return md5.hexdigest()

In [8]:
#练习：设计一个验证用户登录的函数，根据用户输入的口令是否正确，返回true或者false
db = {
    'michael': 'e10adc3949ba59abbe56e057f20f883e',
    'bob': '878ef96e86145580c38c87f0410ad153',
    'alice': '99b1c2188db85afee403b1536010c2c9'
}  

In [12]:
def login(user,password):
    if db.has_key(user):
        print "包含了key"
        return db.get(user) == calc_md5(password)
    else:
        print "没有包含key"
        return False

In [13]:
login('michael','sdsdsd')

包含了key


False

In [14]:
login('csd','dsdsd')

没有包含key


False

摘要算法在很多地方都有广泛的应用。要注意摘要算法不是加密算法，不能用于加密（因为无法通过摘要反推明文），只能用于防篡改，但是它的单向计算特性决定了可以在不存储明文口令的情况下验证用户口令。

# itertools 
<a id='4'></a>

Python的内建模块itertools提供了非常有用的用于操作迭代对象的函数。

In [15]:
#repeat负责把一个元素无限重复下去，如果提供第二个参数就可以限定重复次数
import itertools
ns = itertools.repeat('a',10)
for n in ns:
    print n

a
a
a
a
a
a
a
a
a
a


无限序列只有在for迭代时才会无限地迭代下去，如果只是创建了一个迭代对象，它不会事先把无限个元素生成出来，事实上也不可能在内存中创建无限多个元素。

无限序列虽然可以无限迭代下去，但是通常我们会通过takewhile()等函数根据条件判断来截取出一个有限的序列

In [16]:
natuals = itertools.count(1)
ns = itertools.takewhile(lambda x : x<= 10, natuals)
for n in ns:
    print n

1
2
3
4
5
6
7
8
9
10


### groupby() 
groupby()把迭代器中响铃的重复元素挑出来放在一起

In [17]:
for key,group in itertools.groupby('AAAAABBBBCCCCAAADD'):
    print key,list(group)

A ['A', 'A', 'A', 'A', 'A']
B ['B', 'B', 'B', 'B']
C ['C', 'C', 'C', 'C']
A ['A', 'A', 'A']
D ['D', 'D']


实际上挑选规则是通过函数完成的，只要作用于函数的两个元素返回的值相等，这两个元素就被认为是在一组的，而函数返回值作为组的key。如果我们要忽略大小写分组，就可以让元素'A'和'a'都返回相同的key：

In [18]:
for key,group in itertools.groupby('AaaBBbcC',lambda c:c.upper()):
    print key,list(group)

A ['A', 'a', 'a']
B ['B', 'B', 'b']
C ['c', 'C']


### imap()
imap和map的区别时imap可以作用于无穷序列，并且，如果两个序列长度的禅古的不一致，以短的那个为准

In [20]:
for x in itertools.imap(lambda x,y : x*y,[10,20,30],itertools.count(1)):
    print x

10
40
90


# XML
<a id='5'></a>

操作XML有两种方法，DOM和SAX，DOM会把整个XML读入内存，解析为树，因此占用内存大，解析慢，有点事可以任意遍历树的节点。SAX是流模式，边读边解析，占用内存小，解析快，缺点是我们需要自己处理时间。

正常情况下，优先考虑SAX，因为DOM实在是太占内存。

在Python中使用SAX解析XML非常简介，通常我们关心的事件是start_element,end_element,char_data。准备好这三个函数就可以解析XML了


In [22]:
from xml.parsers.expat import ParserCreate

class DefaultSaxHandler(object):
    def start_element(self,name,attrs):
        print ('sax: start_element: %s, attrs: %s' %(name,str(attrs)))
        
    def end_element(self,name):
        print ('sax: end_element: %s' %name)
        
    def char_data(self,text):
        print ('sax: char_data: %s' %text)
        
xml = r'''<?xml version="1.0"?>
<ol>
    <li><a href="/python">Python</a></li>
    <li><a href="/ruby">Ruby</a></li>
</ol>
'''

In [26]:
handler = DefaultSaxHandler()
parser = ParserCreate()
parser.returns_unicode = True
parser.StartElementHandler = handler.start_element
parser.EndElementHandler = handler.end_element
parser.CharacterDataHandler = handler.char_data
parser.Parse(xml)

sax: start_element: ol, attrs: {}
sax: char_data: 

sax: char_data:     
sax: start_element: li, attrs: {}
sax: start_element: a, attrs: {u'href': u'/python'}
sax: char_data: Python
sax: end_element: a
sax: end_element: li
sax: char_data: 

sax: char_data:     
sax: start_element: li, attrs: {}
sax: start_element: a, attrs: {u'href': u'/ruby'}
sax: char_data: Ruby
sax: end_element: a
sax: end_element: li
sax: char_data: 

sax: end_element: ol


1

# HTMLParser
<a id='6'></a>

如果我们要编写一个搜索引擎，第一步是用爬虫把目标网站的页面抓取下来，第二部时解析该HTML页面，看看里面的内容到底是新闻、图片还是视频。

假如第一步已经完成，第二步就该解析HTML了，HTML本质上时XML的子集，但是HTML的语法没有XML那么严格，所以不能用标准的DOM或者SAX来解析HTML

Python提供了HTMLParser来方便的解析HTML，只需要简单几行代码：

In [30]:
from HTMLParser import HTMLParser
from htmlentitydefs import name2codepoint

class MyHTMLParser(HTMLParser):
    def handle_starttag(self,tag,attrs):
        print('</%s>' %tag)
        
    def handle_endtag(self,tag):
        print('<%s/>' %tag)
        
    def handle_startendtag(self,tag,attrs):
        print('<%s/>' %tag)
        
    def handle_data(self,data):
        print('data: %s' %data)
        
    def handle_comment(self,data):
        print('<!-- -->')
        
    def handle_entityref(self,name):
        print('&%s;' %name)
        
    def handle_charref(self,name):
        print('&#%s;' %name)
        

In [31]:
parser = MyHTMLParser()
parser.feed('<html><head></head><body><p>Some <a href=\"#\">html</a> tutorial...<br>END</p></body></html>')

</html>
</head>
<head/>
</body>
</p>
data: Some 
</a>
data: html
<a/>
data:  tutorial...
</br>
data: END
<p/>
<body/>
<html/>


In [33]:
from HTMLParser import HTMLParser
import urllib

class PyEventParser(HTMLParser):

    def __init__(self):
        HTMLParser.__init__(self)
        self._count = 0
        self._events = dict()
        self._flag = None

    def handle_starttag(self, tag, attrs):
        if tag == 'h3' and attrs.__contains__(('class', 'event-title')):
            self._count += 1
            self._events[self._count] = dict()
            self._flag = 'event-title'
        if tag == 'time':
            self._flag = 'time'
        if tag == 'span' and attrs.__contains__(('class', 'event-location')):
            self._flag = 'event-location'

    def handle_data(self, data):
        if self._flag == 'event-title':
            self._events[self._count][self._flag] = data
        if self._flag == 'time':
            self._events[self._count][self._flag] = data
        if self._flag == 'event-location':
            self._events[self._count][self._flag] = data
        self._flag = None

    def event_list(self):
        print '近期关于Python的会议有：', self._count, '个，具体如下：'
        for event in self._events.values():
            print event['event-title'], '\t', event['time'], '\t', event['event-location']


try:
    parser = PyEventParser()
    pypage = urllib.urlopen('https://www.python.org/events/python-events/')
    pyhtml = pypage.read()
except IOError,e:
    print 'IOError:', e
else:
    parser.feed(pyhtml)
    parser.event_list()
finally:
    pypage.close()

近期关于Python的会议有： 8 个，具体如下：
PyDays Vienna 2017 	05 May  	FH Technikum Wien, Höchstädtplatz 6, 1200 Vienna, Austria
PyData London 2017 	05 May  	 Bloomberg No 39/45 Finsbury Square City Gate House, EC2A 1 PQ London, United Kingdom
GeoPython 2017 	08 May  	Basel, Switzerland
PyCon US 2017 	17 May  	Portland, Oregon, USA
PyDataBCN 2017 	19 May  	Barcelona, Spain
PyConWEB 2017 	27 May  	Munich, Germany
DjangoCon Europe 2017 	03 April  	Florence, Italy
Rencontres Django 2017 	01 April  	Toulon, France


# 图形编程 
<a id='7'></a>

### PIL模块 

要详细了解PIL的强大功能，请请参考PIL官方文档：
http://effbot.org/imagingbook/

In [2]:
import Image, ImageFilter
import os

In [7]:
print os.path

<module 'ntpath' from 'C:\Users\haier-003\Anaconda2\lib\ntpath.pyc'>


In [8]:
f = open('data.txt','r')
f.read()

'\nhello,world!'

In [9]:
f.close()

In [12]:
im = Image.open('test.jpg')
w, h = im.size
#缩放到百分之五十
im.thumbnail((w//2, h//2))
im.save('thumbnail.jpg','jpeg')

### 生成验证码图片

In [19]:
import Image, ImageDraw, ImageFont, ImageFilter
import random

#随机字母：
def rndChar():
    return chr(random.randint(65,90))

#随机颜色1：
def rndColor1():
    return(random.randint(64, 255),random.randint(64, 255),random.randint(64, 255))

def rndColor2():
    return(random.randint(23, 127),random.randint(23, 127),random.randint(23, 127))

#240 x 60
width = 60*4
height = 60
image = Image.new('RGB',(width,height),(255,255,255))

#创建Font对象：
font = ImageFont.truetype('Arial.ttf',36)
#创建Draw对象：
draw = ImageDraw.Draw(image)
#填充每个像素：
for x in range(width):
    for y in range(height):
        draw.point((x,y), fill=rndColor1())
#输出文字：
for t in range(4):
    draw.text((60 * t+10, 10),rndChar(), font=font,fill=rndColor2())
    
#模糊：
image = image.filter(ImageFilter.BLUR)
image.save('code.jpg', 'jpeg')


### 图形界面

Python支持多种图形界面的第三方库，包括：
- TK
- wxWidgets
- Qt
- GTK
但是Python自带的库是支持Tkinter，使用Tkinter无需安装任何包，就可以直接使用

### 第一个GUI程序 

In [1]:
from Tkinter import *

In [5]:
#从Frame派生一个Application类，这是所有Widget的父容器
class Application(Frame):
    def __init__(self, master=None):
        Frame.__init__(self,master)
        self.pack()
        self.createWidgets()
        
    def createWidgets(self):
        self.helloLabel = Label(self, text='Hello,world!')
        self.helloLabel.pack()
        self.quitButton = Button(self,text='Quit', command=self.quit)
        self.quitButton.pack()

在GUI中，每个Button、Label、输入框等，都是一个Widget。Frame则是可以容他其他Widget的Widget，所有的Widget组合起来就是一颗树

pack()方法把Widget加入到父容器中，并实现布局。pack()是最简单的布局，grid()可以实现更复杂的布局。

在createWidgets()方法中，我们创建一个Label和一个Button，当Button被点击时，出发self.quit()使程序退出

第三步，实例化Application，并启动消息循环:

In [3]:
app = Application()
#设置窗口标题：
app.master.title('Hello World')
#主消息循环：
app.mainloop()

In [6]:
#加入文本框
from Tkinter import *
import tkMessageBox

class Application(Frame):
    def __init__(self,master=None):
        Frame.__init__(self, master)
        self.pack()
        self.createWidgets()

    def createWidgets(self):
        self.nameInput = Entry(self)
        self.nameInput.pack()
        self.alertButton = Button(self, text='Hello', command=self.hello)
        self.alertButton.pack()

    def hello(self):
        name = self.nameInput.get() or 'world'
        tkMessageBox.showinfo('Message', 'Hello, %s' %name)

In [8]:
app = Application()
#设置窗口标题：
app.master.title('Hello World')
#主消息循环：
app.mainloop()